# 1st Notebook: Prepare the training samples from raw data

&nbsp;

The <strong>data treatment</strong>, previous to the network training, it is a very important part when it comes to Machine Learning analyses. There are infinite ways to represent the given information and find the optimal one is often tricky.

It is also important to point out that the performance of the architecture will have a huge dependence on how the samples are presented.



## 1. Data format


### 1.1 Raw Data

The events available for the challenge are given in a .csv file with all the information needed for the DNN training. With the exception of the heading, each row corresponds to a single event of the sample.

Since we are working with Keras (and Keras is built with NumPy) we will need to transform the rows of the CSV file into NumPy arrays that the DNN can read.

### 1.2 Reading the data

The package used to read the data first is [pandas](https://pandas.pydata.org/).

By means of the ```read_csv(file, delimiter)``` method we can load all the information of the CSV file into a ```pandas.DataFrame``` object that we can access easily.

We can take a look then to the data:
   

In [1]:
import pandas

data = pandas.read_csv("training.csv", delimiter = ',') # DataFrame object with al the info of the csv

data.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


And explore a little bit the information we have i.e. number of events, variables... etc.

In [2]:
### Print general information about the data:
print ('Size of data: {}'.format(data.shape))
print ('Number of events: {}'.format(data.shape[0]))
print ('Number of columns: {}'.format(data.shape[1]))

print ('\nList of features in dataset:')

for col in data.columns:
    print (col)

Size of data: (250000, 33)
Number of events: 250000
Number of columns: 33

List of features in dataset:
EventId
DER_mass_MMC
DER_mass_transverse_met_lep
DER_mass_vis
DER_pt_h
DER_deltaeta_jet_jet
DER_mass_jet_jet
DER_prodeta_jet_jet
DER_deltar_tau_lep
DER_pt_tot
DER_sum_pt
DER_pt_ratio_lep_tau
DER_met_phi_centrality
DER_lep_eta_centrality
PRI_tau_pt
PRI_tau_eta
PRI_tau_phi
PRI_lep_pt
PRI_lep_eta
PRI_lep_phi
PRI_met
PRI_met_phi
PRI_met_sumet
PRI_jet_num
PRI_jet_leading_pt
PRI_jet_leading_eta
PRI_jet_leading_phi
PRI_jet_subleading_pt
PRI_jet_subleading_eta
PRI_jet_subleading_phi
PRI_jet_all_pt
Weight
Label


We know that ```EventId```, ```Weight``` and ```Label``` are not training features but control variables, because they have no physical information. Therefore, we exclude them in the NumPy array creation process:

In [3]:
### Select the features with physical information:
features = [h for h in list(data) if h not in ['EventId', 'Weight', 'Label']] 

>**Note: -999.0 values are missing or meaningless inputs in the data. They do not provide any physical information. For example, in any event with only one jet ```PRI_jet_subleading_pt```, ```PRI_jet_subleading_eta``` or ```PRI_jet_subleading_phi``` would have -999.0 value since there is no jet  where the kinematics can be obtained from.**

## 3. Physical features representation

Before training the network, it is very important to study the physical features we are going to use. This information could give us ideas of how to represent the data or even design the ML algorithm. 

The distributions of all the physical features are studied. We import the [NumPy](http://www.numpy.org/) package to use ```numpy.array``` structures and also [Matplotlib](https://matplotlib.org/) to generate the histograms.

One histogram is created for each physical variable, with signal and background events separately. All -999.0 values are excluded from the plotting since they do not provide any physical nor relevant information. All histograms are saved in the ```Histograms/``` folder.

In [4]:
import numpy as np
import matplotlib.pyplot as plt


### Loop over the features
for feature in features:
    
    print(">>> Plotting " + feature + " histogram...")
    
    # Signal and background values
    signal_values = list( data.loc[data['Label'] == 's', feature] )
    background_values = list( data.loc[data['Label'] == 'b', feature] )
    
    signal_values = list(filter(lambda x: x != -999.0, signal_values))
    background_values = list(filter(lambda x: x != -999.0, background_values))
    
    # Define the histogram binning
    xmin = min(signal_values + background_values) 
    xmax = max(signal_values + background_values)
    binning = np.linspace(xmin, xmax, 61) 
    
    # Plot and save the histogram
    plt.clf()
    fig = plt.figure(figsize =(6,6))
    ax = fig.add_subplot(111)
    ax.margins(x = 0)
    
    ax.hist(background_values, bins = binning, color = 'k', alpha = 0.4, histtype = 'stepfilled', 
             linewidth = 1, edgecolor = 'k', label = 'Background', density = True)
    ax.hist(signal_values, bins = binning, color = 'r', alpha = 0.3, histtype = 'stepfilled', 
             linewidth = 1, edgecolor = 'r', label = 'Signal', density = True)
    ax.legend(loc = 'best', fontsize = 10)
    ax.set_xlabel(feature, fontsize = 14)
    ax.set_ylabel('Event density', fontsize = 14)
    ax.tick_params(axis='both', which='both', direction='in', 
                   bottom=True, top=True, left=True, right=True, labelsize=12)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText = True)
    fig.savefig('Histograms/'+feature+'_histo.png', dpi = 600)
          
    print("    Done")

>>> Plotting DER_mass_MMC histogram...
    Done
>>> Plotting DER_mass_transverse_met_lep histogram...
    Done
>>> Plotting DER_mass_vis histogram...
    Done
>>> Plotting DER_pt_h histogram...
    Done
>>> Plotting DER_deltaeta_jet_jet histogram...
    Done
>>> Plotting DER_mass_jet_jet histogram...
    Done
>>> Plotting DER_prodeta_jet_jet histogram...
    Done
>>> Plotting DER_deltar_tau_lep histogram...
    Done
>>> Plotting DER_pt_tot histogram...
    Done
>>> Plotting DER_sum_pt histogram...
    Done
>>> Plotting DER_pt_ratio_lep_tau histogram...
    Done
>>> Plotting DER_met_phi_centrality histogram...
    Done
>>> Plotting DER_lep_eta_centrality histogram...
    Done
>>> Plotting PRI_tau_pt histogram...
    Done
>>> Plotting PRI_tau_eta histogram...
    Done
>>> Plotting PRI_tau_phi histogram...
    Done
>>> Plotting PRI_lep_pt histogram...
    Done
>>> Plotting PRI_lep_eta histogram...
    Done
>>> Plotting PRI_lep_phi histogram...
    Done
>>> Plotting PRI_met histogram...


/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


    Done
>>> Plotting PRI_met_phi histogram...
    Done
>>> Plotting PRI_met_sumet histogram...
    Done
>>> Plotting PRI_jet_num histogram...
    Done
>>> Plotting PRI_jet_leading_pt histogram...
    Done
>>> Plotting PRI_jet_leading_eta histogram...
    Done
>>> Plotting PRI_jet_leading_phi histogram...
    Done
>>> Plotting PRI_jet_subleading_pt histogram...
    Done
>>> Plotting PRI_jet_subleading_eta histogram...
    Done
>>> Plotting PRI_jet_subleading_phi histogram...
    Done
>>> Plotting PRI_jet_all_pt histogram...
    Done


## 4. From raw samples to training datasets

In this part of the notebook, it is explained how to get the samples ready for the training. The steps are the following:

- **4.1**  Definition of train, development and test datasets

- **4.2**  Normalization

- **4.3**  NumPy array creation

- **4.4**  Class balance

- **4.5**  Shuffling

- **4.6**  Save variables

### 4.1 Definition of train and test datasets

The first thing to do is select the events that are going to be used for training the DNN (train dataset) and testing its final performance once trained (test dataset).

Since both development and test datasets are used to evaluate the DNN performance, it is mandatory to have a proper class balance in them i.e. having the same number of signal and background events. If not, the results obtained from those datasets would not reflect the real DNN efficiency.

Let's see how many events of each kind are stored in the csv:

In [5]:
# Get two separate pandas dataframes for signal and background events:
signal_data = data.loc[data['Label'] == 's']
bkg_data = data.loc[data['Label'] == 'b']

print ('Number of total signal events: {}'.format(signal_data.shape[0]))
print ('Number of total background events: {}'.format(bkg_data.shape[0]))

Number of total signal events: 85667
Number of total background events: 164333


While a total of 164333 samples are background events, only 85667 events are hadronic decays of the Higgs boson. This means that we will have only ~34% signal events of the entire dataset for training and testing the DNN.

As said above, both development and test datasets must have 50%-50% signal-background balance to obtain reliable results and (for the same reason) no event must be duplicated there. We build those datasets first by taking 5000 events of each kind for each one of them. The remaining events will compoung the train dataset.

Each dataset will have associated one ```pandas.DataFrame``` object:

In [6]:
# Save 5000 events from each class for both the development and test datasets:
n_test = 5000

signal_test = signal_data[:n_test]
bkg_test = bkg_data[:n_test] 
test = pandas.concat([signal_test, bkg_test]) # final test dataset

# The remaining events will compound the training dataframe:

signal_train = signal_data[n_test:]
bkg_train = bkg_data[n_test:]
train = pandas.concat([signal_train, bkg_train]) # final train dataset

### 4.2 Normalization

Having values of different features taking wildly different ranges could make the learning process harder. A good practice is to do the so-called **feature-wise normalization**, forcing the feature distributions to be centered at 0 while having standard deviations equal to the unity.

To have the values properly normalized we could just substract the mean value for each feature, and divide by its standard deviation. So, for each sample <em>i</em> and feature <em>j</em> value $x^{(i)}_j$ we apply:

$$ x'^{(i)}_j = \dfrac{x^{(i)}_j - \bar{x}_j}{\sigma (x_j)}$$

being the mean value $\bar{x}_j$ of feature *f* defined as

$$\bar{x}_j = \dfrac{1}{N}\sum_{i = 1}^{N} x^{(i)}_j$$

and its standard deviation $\sigma (x_j)$ defined as 

$$\sigma(x_j) = \sqrt{\dfrac{\sum_{i = 1}^{N} (x^{(i)}_j - \bar{x}_j)}{N-1}}$$  

---
---

> In addition, there are some **IMPORTANT FACTS TO REMEMBER IF THE INPUT IS NORMALIZED** that must be rigurously taken into account:
> - Both $\bar{x}_j$ and $\sigma (x_j)$ values **must be computed ONLY with the samples of the train dataset**. Remember that development and test datasets are just to measure performance and should not be used for nothing more.
> - Although $\bar{x}_j$ and $\sigma (x_j)$ are computed with the train dataset, **all the datasets must be normalized in the same way**. The DNN is trained to classify samples of the same kind of those that have been used to compute its weights.
> - -999.0 values are meaningless or missing inputs. The DNN is supossed to learn by itself that this value means exactly that, and must be the same for each feature. For these reason, **they must not enter in the $\bar{x}_j$ and $\sigma (x_j)$ computations nor be normalized either**. They just have to be left this way.

We run over the events of the train dataset and we save the meand and standar deviation values for each one of the features in a python dictionary:

In [7]:
# Initialization:
means = {} # dict with the means of every feature
stds = {} # dict with the standard deviations of every feature

for feature in features:
    true_values = train.loc[data[feature] != -999.0, feature] # We exclude -999.0 values
    means[feature] = true_values.values.mean()
    stds[feature] = true_values.values.std()

### 4.3 Input creation

The DNN will take plain vectors $\vec{x}^{(i)}$ as an input. These are given to the network as a matrix of the form

$$\vec{X} = 
\begin{pmatrix}
\vec{x}^{(1)} \\
\vec{x}^{(2)} \\
... \\
\vec{x}^{(\text{nEventos})}
\end{pmatrix}$$

in which each one of the $x^{(i)}$ vectors is identified with an event of the train dataset. Therefore, the columns of this matrix will be asocciated with one feature each. That is 

$$\vec{X} = 
\begin{pmatrix}
\vec{x}^{(1)}_1 & \vec{x}^{(1)}_2 & ... & \vec{x}^{(1)}_{\text{nFeatures}} \\
\vec{x}^{(2)}_1 & \vec{x}^{(2)}_2 & ... & \vec{x}^{(2)}_{\text{nFeatures}} \\
... & ... & ...& ... \\
\vec{x}^{(\text{nEvents})}_1 & \vec{x}^{(\text{nEvents})}_2 & ... &\vec{x}^{(\text{nEvents})}_{\text{nFeatures}} \\
\end{pmatrix}$$ 

  
  
Appart from the sample matrices $\vec{X}$ we will need another vector $\vec{y}$ to let the DNN know if each one of the $\vec{x}^{(i)}$ vectors corresponds with a signal or a background event. This vector is called label vector and its elements will be either 0 (for background events) or 1 (for signal events):

$$\vec{y} = 
\begin{pmatrix}
y_0 \\
y_1 \\
... \\
y_{(\text{nEvents})}
\end{pmatrix}$$

These two ingredients, sample matrix $\vec{X}$ an label vector $\vec{y}$, are the inputs of the network, and we will need to create them for train, develop and test datasets.

---
---

Since Keras is build with NumPy, the vectors will need to be defined as ```numpy.array``` objects. To create them, we define first a python function that will receive as an input a ```pandas.DataFrame``` object with the samples, a list with the features we want to include, and two python dictioraries with the mean and standard deviation values (created in the step before). The output of this function will be two ```numpy.array```'s: the sample matrix and the label vector, already normalized.

We use this function to create the inputs of the train, development and test dataset.


> **Note** that this function contains a loop that runs over all the events in the ```pandas.DataFrame``` given as the input so it will take time to create the vectors. (~1 hour and a half)

In [8]:
import numpy as np

def getXandY(df, fnames, mean_values, std_values):

    """
    Returns the sample matrix X and the label vector y constructed with:
    - The events of df (pandas.DataFrame)
    - The features indicated in fnames (list)
    - The mean indicated in mean_values (dictionary)
    - The standard deviation indicated in std_values (dictionary)
    """

    progress = [int(i*float(len(df.index.values))) for i in np.linspace(0, 1, 101)]
    
    X = np.zeros(shape = (df.shape[0], len(fnames))) # empty sample matrix X (to be filled)
    Y = np.zeros(shape = (df.shape[0], 1)) # empty label vector y (to be filled)

    # Loop over dataset
    for i,idx in enumerate(df.index.values):
        for j,feature in enumerate(fnames):
            
            # (i: event)
            # (j: feature)
        
            # X filling:
            value = df.loc[idx][feature]
            if (value != -999.0): 
                X[i][j] = (value - mean_values[feature])/std_values[feature]
            else: 
                X[i][j] = -5.0
            
            # y_test filling:
            if df.loc[idx]['Label'] == 's':
                Y[i][0] = 1
            else:
                Y[i][0] = 0
                
        if (i in progress):
            print("Progress: "+ str(i)+"/"+str(len(df.index.values)) +" completed")
                
    return X,Y

In [10]:
# Train dataset:
x_train, y_train = getXandY(train, features, means, stds)

Progress: 0/240000 completed
Progress: 2400/240000 completed
Progress: 4800/240000 completed
Progress: 7200/240000 completed
Progress: 9600/240000 completed
Progress: 12000/240000 completed
Progress: 14400/240000 completed
Progress: 16800/240000 completed
Progress: 19200/240000 completed
Progress: 21600/240000 completed
Progress: 24000/240000 completed
Progress: 26400/240000 completed
Progress: 28800/240000 completed
Progress: 31200/240000 completed
Progress: 33600/240000 completed
Progress: 36000/240000 completed
Progress: 38400/240000 completed
Progress: 40800/240000 completed
Progress: 43200/240000 completed
Progress: 45600/240000 completed
Progress: 48000/240000 completed
Progress: 50400/240000 completed
Progress: 52800/240000 completed
Progress: 55200/240000 completed
Progress: 57600/240000 completed
Progress: 60000/240000 completed
Progress: 62400/240000 completed
Progress: 64800/240000 completed
Progress: 67200/240000 completed
Progress: 69600/240000 completed
Progress: 72000/24

In [11]:
# Test dataset:
x_test, y_test = getXandY(test, features, means, stds) 

Progress: 0/10000 completed
Progress: 100/10000 completed
Progress: 200/10000 completed
Progress: 300/10000 completed
Progress: 400/10000 completed
Progress: 500/10000 completed
Progress: 600/10000 completed
Progress: 700/10000 completed
Progress: 800/10000 completed
Progress: 900/10000 completed
Progress: 1000/10000 completed
Progress: 1100/10000 completed
Progress: 1200/10000 completed
Progress: 1300/10000 completed
Progress: 1400/10000 completed
Progress: 1500/10000 completed
Progress: 1600/10000 completed
Progress: 1700/10000 completed
Progress: 1800/10000 completed
Progress: 1900/10000 completed
Progress: 2000/10000 completed
Progress: 2100/10000 completed
Progress: 2200/10000 completed
Progress: 2300/10000 completed
Progress: 2400/10000 completed
Progress: 2500/10000 completed
Progress: 2600/10000 completed
Progress: 2700/10000 completed
Progress: 2800/10000 completed
Progress: 2900/10000 completed
Progress: 3000/10000 completed
Progress: 3100/10000 completed
Progress: 3200/10000

### 4.4 Class balance

There is one important thing to take into account when dealing with ML problems, which is the class balance in the datasets. It was mentioned before that development and test datasets must have the same number of signal/background events to give reliable measurements of performance.

For example, suppose that a bad trained DNN is very efficient in identifying signal events but classifies background events randomly. If its performance is measured with a dataset with 80% signal events our (mistaken) perception would be that this DNN is correctly trained. 

And for the same reason a dataset used to measure the DNN performance must not have duplicates.

Class balance in development and test datasets is adressed by taken the same number of signal and background events from the available data. But what happen with the train dataset which is used to optimize the DNN that takes the remaining events? 

Let's see how is the class balance there:

In [ ]:
# Print class balance in training set:
s_train = np.count_nonzero(y_train == 1.0)
b_train = np.count_nonzero(y_train == 0.0)
print("Number of signal events in training set: " + "{}".format(s_train))
print("Number of background events in training set: " + "{}".format(b_train))

As it can be seen, the number of signal events is much lower than the number of background events. Class unbalance in the train dataset is not as dramatic as in development and test ones but could affect the learning process. It may cause, among other things, that the network only learns how to classify one kind of event. In an extreme scenario, if a train dataset is full of events of one kind, the DNN would learn that the "optimal" way to have a good score is to always classify any given event in the most populated category.

The best scenario is having a proper 50-50 class balance. There are to valid options to get it:
- Change DNN configuration to apply a weight in the less populated class when training
- Duplicate some events until matching the most populated class number

The last one is the followed one in this notebook:

In [ ]:
x_train_extra = []

while(s_train < b_train): # repeat until we have our classes balanced
    
    for i,idx in enumerate(signal_train.index.values):
        
        if s_train >= b_train: break # if we have all the extra arrays we need we just quit
        
        x_extra = np.zeros(len(features))
        
        for j,feature in enumerate(features):
            
            value = signal_train.loc[idx][feature]
            x_extra[j] = (value - means[feature])/stds[feature]
            
        x_train_extra.append(x_extra)
        s_train += 1
        
### Add this extra vectors to the existing training dataset:
x_train = np.concatenate((x_train, np.array(x_train_extra)), axis = 0)
y_train = np.concatenate((y_train, np.full(shape = (len(x_train_extra), 1), fill_value = 1.0)))       

And if we check again the number of events of each class in the train dataset we can see that now it is properly balanced:

In [ ]:
# Print again class balance in training set:
s_train = np.count_nonzero(y_train == 1.0)
b_train = np.count_nonzero(y_train == 0.0)
print("Number of signal events in training set: " + "{}".format(s_train))
print("Number of background events in training set: " + "{}".format(b_train))

### 4.5 Shuffle the samples

Finally, when we have our datasets prepared, it is mandatory to assure that the input in the DNN is correctly shuffled. Events of different classes have to be read by the DNN in an homogeneous way:

In [ ]:
import random

### Get the indexes of each dataset:
idx_train = np.arange(y_train.shape[0])
idx_test = np.arange(y_test.shape[0])

### Shuffle them:
random.shuffle(idx_train)
random.shuffle(idx_test)

### And apply the new ordering to every input of each dataset:
y_train = y_train[idx_train]
x_train = x_train[idx_train]
y_test = y_test[idx_test]
x_test = x_test[idx_test]

### 4.6 Save the variables

As the sample creation process is long and also computationally expensive, it not advisable to create the inputs every time we want to train the DNN.

For this reason, input variables are created once and saved in a pickle file that will allow to access them in the future.

In [ ]:
import pickle

with open('DNNtraining_variables2.p', 'wb') as file_:
        pickle.dump([x_train, y_train, x_test, y_test], file_)